# Wrappers example

We will see on this notebook what are the wrappers defined by sinergym and how to use them. Currently, we have developed a **normalization wrapper**,
**multi-observation wrapper** and **Logger wrapper**

In [13]:
import gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)
from sinergym.utils.common import RANGES_5ZONE

In [14]:
env_with_wrappers = MultiObsWrapper(LoggerWrapper(NormalizeObservation(env=gym.make('Eplus-demo-v1'),ranges=RANGES_5ZONE)))

[2022-05-16 13:51:12,674] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:12,674] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:12,674] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:12,674] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:12,674] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:12,674] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:12,678] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up extra configuration in buildin

In [15]:
env_with_wrappers = NormalizeObservation(gym.make('Eplus-demo-v1'), ranges=RANGES_5ZONE)

[2022-05-16 13:51:13,714] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:13,714] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:13,714] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:13,714] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:13,714] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:13,714] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:13,714] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeri

In [16]:
env_with_wrappers = LoggerWrapper(NormalizeObservation(gym.make('Eplus-demo-v1'), ranges=RANGES_5ZONE))

[2022-05-16 13:51:14,830] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:14,830] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:14,830] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:14,830] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:14,830] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:14,830] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-05-16 13:51:14,830] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeri

.. warning:: The order of wrappers if you are going to use several at the same time is really important.
             The correct order is **Normalization - Logger - MultiObs** and subsets (for example, *Normalization* - *Multiobs* is valid).

.. note:: For more information about Sinergym Logger, visit :ref:`Logger`.

Now we just simply use the environment with the wrappers, for example:

In [17]:
for i in range(1):
    obs = env_with_wrappers.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env_with_wrappers.action_space.sample()
        obs, reward, done, info = env_with_wrappers.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env_with_wrappers.close()

[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:14,990] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-05-16 13:51:15,005] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/

Reward:  -0.6725497890372467 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 3133.716390651039, 'total_power_no_units': -0.3133716390651039, 'comfort_penalty': -1.0317279390093894, 'temperatures': [18.96827206099061], 'out_temperature': 1.8, 'action_': [17.0, 28.0]}
Reward:  -1809.6247298837213 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 5107.400520896434, 'total_power_no_units': -0.5107400520896433, 'comfort_penalty': -2.40452930117074, 'temperatures': [17.59547069882926], 'out_temperature': -7.0, 'action_': [15.0, 30.0]}
Reward:  -3564.6143288880503 {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'total_power': 4732.071629972534, 'total_power_no_units': -0.4732071629972534, 'comfort_penalty': -0.0, 'temperatures': [20.94123714194632], 'out_temperature': 8.1, 'action_': [21.0, 24.0]}
Reward:  -4753.266089024571 {'timestep': 8640, '

[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-05-16 13:51:35,848] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
